In [166]:
import pandas as pd
import numpy as np 
import plotly.express as ex 
import plotly.graph_objects as go

# 数据清洗&预处理

In [417]:
#加载三张数据集
user_df = pd.read_csv('databases/user—test.csv',encoding='gb2312')
order_df = pd.read_csv('databases/Order-test.csv')
consult_df = pd.read_csv('databases/consult-test.csv')

## 处理 User 表格

In [418]:
print(user_df.info(),"\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870460 entries, 0 to 870459
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         870460 non-null  int64  
 1   sex             62753 non-null   object 
 2   age             61970 non-null   float64
 3   country         866918 non-null  object 
 4   province_name   866918 non-null  object 
 5   city_name       866918 non-null  object 
 6   city_level      863913 non-null  object 
 7   is_member_flag  870460 non-null  int64  
 8   first_day       811277 non-null  object 
 9   last_day        870460 non-null  object 
 10  be_member_time  596011 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 73.1+ MB
None 



In [419]:
# 这里 sex & age Missing value 是80w+, 这个数据集总共才87W, 缺失值总共约占数据集92%
# 直接去掉这两行
user_df = user_df.drop(['sex','age'],axis=1)

In [420]:
# 查看province_name 为缺失值有3542个，由于省名缺失，对后续分析有影响。
# 所以,直接将这三千多行数据去掉，约占总数据集的0.4%，影响不大
# user_df = user_df.dropna(subset=['province_name'])
# columnlist = user_df.columns.values
print(user_df.isna().sum())
print(user_df.shape)

user_id                0
country             3542
province_name       3542
city_name           3542
city_level          6547
is_member_flag         0
first_day          59183
last_day               0
be_member_time    274449
dtype: int64
(870460, 9)


In [421]:
#缺失city_level这些地方属于一些特殊的省直管县级市，按照这种方法，使用‘其他城市’去替换Nan
user_df['city_level'] = user_df['city_level'].fillna("其他城市")

In [422]:
#检测是否有重复row
user_df.duplicated().sum()


np.int64(0)

In [423]:
user_df['first_day'] = pd.to_datetime(user_df['first_day'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')
user_df['last_day'] = pd.to_datetime(user_df['last_day'])
user_df['be_member_time'] = pd.to_datetime(user_df['be_member_time'])

user_df.head(5)

,user_id,country,province_name,city_name,city_level,is_member_flag,first_day,last_day,be_member_time
0,570009353210116,中国,山东,潍坊,三线城市,1,2023-04-03 00:00:00,2023-08-02,2023-04-03 19:42:00
1,570020406530107,中国,贵州,六盘水,四线城市,1,2023-03-09 00:00:00,2023-03-09,2023-03-09 19:52:00
2,570020458000111,中国,广东,广州,一线城市,1,2023-06-25 00:00:00,2023-06-29,2023-06-29 12:38:00
3,570020914800114,中国,山东,青岛,新一线城市,1,2023-06-25 00:00:00,2023-07-02,2022-04-20 10:22:00
4,570021004330160,中国,江西,上饶,三线城市,1,2023-05-11 00:00:00,2023-07-09,2022-04-22 10:17:00


In [424]:
#关于填补first_day
user_df['first_day'] = user_df['first_day'].fillna(user_df['be_member_time'])
user_df.reset_index(drop=True)

,user_id,country,province_name,city_name,city_level,is_member_flag,first_day,last_day,be_member_time
0,570009353210116,中国,山东,潍坊,三线城市,1,2023-04-03 00:00:00,2023-08-02,2023-04-03 19:42:00
1,570020406530107,中国,贵州,六盘水,四线城市,1,2023-03-09 00:00:00,2023-03-09,2023-03-09 19:52:00
2,570020458000111,中国,广东,广州,一线城市,1,2023-06-25 00:00:00,2023-06-29,2023-06-29 12:38:00
3,570020914800114,中国,山东,青岛,新一线城市,1,2023-06-25 00:00:00,2023-07-02,2022-04-20 10:22:00
4,570021004330160,中国,江西,上饶,三线城市,1,2023-05-11 00:00:00,2023-07-09,2022-04-22 10:17:00
...,...,...,...,...,...,...,...,...,...
870455,570072597650156,中国,重庆,重庆,新一线城市,0,2023-06-30 00:00:00,2023-06-30,NaT
870456,570072523900118,中国,山东,潍坊,三线城市,0,2023-06-30 00:00:00,2023-08-03,NaT
870457,570072538870133,中国,黑龙江,大庆,三线城市,1,2023-06-30 00:00:00,2023-06-30,2023-06-30 19:08:00
870458,570072510060139,中国,陕西,西安,新一线城市,1,2023-06-30 00:00:00,2023-06-30,2023-06-30 21:03:00


In [428]:
user_df[user_df['user_id']==570026463920104]

,user_id,country,province_name,city_name,city_level,is_member_flag,first_day,last_day,be_member_time
17,570026463920104,中国,黑龙江,哈尔滨,二线城市,1,2022-08-16 14:37:00,2023-07-18,2022-08-16 14:37:00


In [411]:
condition1 = user_df['first_day'] > user_df['be_member_time']
user_df = user_df[~condition1]
condition2 = user_df['first_day'] > user_df['last_day']
user_df = user_df[~condition2]
condition3 = user_df['last_day'] < user_df['be_member_time']
user_df = user_df[~condition3]

In [412]:
user_df.shape

(562272, 9)

In [413]:
#增加续费会员的标签
user_df['renew_time'] = (user_df['be_member_time'] + pd.DateOffset(days=365))
user_df['is_renew'] = (user_df['renew_time'] <= user_df['last_day']).astype(int) #判断会员是否续费（会员到期日子早于会员最近一次登陆日期）

In [416]:
print(user_df[['be_member_time', 'first_day']].dtypes)
user_df.head(5)

be_member_time    datetime64[ns]
first_day                 object
dtype: object


,user_id,country,province_name,city_name,city_level,is_member_flag,first_day,last_day,be_member_time,renew_time,is_renew
0,570009353210116,中国,山东,潍坊,三线城市,1,2023-04-03 00:00:00,2023-08-02,2023-04-03 19:42:00,2024-04-02 19:42:00,0
9,570023426180132,中国,北京,北京,一线城市,0,2023-03-04 00:00:00,2023-03-09,NaT,NaT,0
12,570025239650135,中国,上海,上海,一线城市,1,2023-03-08 00:00:00,2023-08-17,2023-07-15 18:25:00,2024-07-14 18:25:00,0
17,570026463920104,中国,黑龙江,哈尔滨,二线城市,1,NaN,2023-07-18,2022-08-16 14:37:00,2023-08-16 14:37:00,0
18,570027668630107,中国,四川,成都,新一线城市,0,2023-03-25 00:00:00,2023-03-25,NaT,NaT,0


In [415]:
user_df['mbr_days'] = (user_df['be_member_time'] - user_df['first_day']).dt.days #普通用户到付费用户转化天数
user_df['mbr_lose_days'] = (user_df['last_day'] - user_df['be_member_time']).dt.days #会员到流失转化天数

/var/folders/4q/j0ysgpnj36545bb_2_dt0tr80000gn/T/ipykernel_67548/2079450255.py:1: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  user_df['mbr_days'] = (user_df['be_member_time'] - user_df['first_day']).dt.days #普通用户到付费用户转化天数


TypeError: unsupported operand type(s) for -: 'Timestamp' and 'str'

In [247]:
#将city_level 转化为数字类型，方便后续分析
def cityLevel(level):
    if level == "一线城市" or level == "新一线城市":
        return 5
    elif level == "二线城市":
        return 4
    elif level == "三线城市":
        return 3
    elif level == "四线城市":
        return 2
    elif level == "五线城市":
        return 1
    else:
        return 0
user_df['city_cate'] = user_df['city_level'].apply(cityLevel)

In [248]:
user_df.to_csv('databases/clean_user_df.csv',encoding='utf-8')

## 处理 Consult 表格

In [249]:
print(consult_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 662158 entries, 0 to 662157
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   consult_no          662158 non-null  int64  
 1   consult_start_time  662158 non-null  object 
 2   consult_end_time    662158 non-null  object 
 3   user_id             662158 non-null  int64  
 4   consult_round       0 non-null       float64
 5   like_level          94404 non-null   float64
dtypes: float64(2), int64(2), object(2)
memory usage: 30.3+ MB
None


In [250]:
# 删除 consult_round 这个variable 全是None 所以可以直接去除
del consult_df['consult_round']

In [251]:
consult_df['consult_start_time'] = pd.to_datetime(consult_df['consult_start_time'],errors='coerce').dt.tz_localize(None)
consult_df['consult_end_time'] = pd.to_datetime(consult_df['consult_end_time'],errors='coerce').dt.tz_localize(None)
consult_df = consult_df[consult_df['consult_start_time'] != consult_df['consult_end_time']] #删除问诊时间异常值

In [252]:
# 计算用户问诊时长
consult_df.loc[:,'duration'] = (consult_df['consult_end_time'] - consult_df['consult_start_time']).dt.seconds/60
consult_df.loc[:, 'duration'] = consult_df['duration'].round()
consult_df['consult_month'] = consult_df['consult_start_time'].dt.to_period('M') #提取问诊时间的月份

In [253]:
# 计算用户最后一次问诊的时间，日记截至2024年元旦，因为此数据集只到2023年底
consult_df['last_consult'] = (pd.to_datetime('2024-01-01') - consult_df['consult_end_time']).dt.days
consult_df['comment'] = consult_df['like_level'].notna().astype(int) #判断用户是否评价
consult_df = consult_df.reset_index(drop='True')
consult_df.to_csv("databases/clean_consult.csv",encoding='utf-8')

## 患者问诊行为特征构建

In [254]:
user_df = pd.read_csv('databases/clean_user_df.csv',encoding='utf-8')
consult_df = pd.read_csv('databases/clean_consult.csv',encoding='utf-8')

In [255]:
user_consult_df = pd.merge(consult_df,user_df,on='user_id',how='left')
del user_consult_df['Unnamed: 0_x']
del user_consult_df['Unnamed: 0_y']

In [256]:
def consult_calcu(df):
    if 'user_id' not in df.columns:
        raise ValueError("Varible user_id Does Not Exist")

    temp = df.groupby('user_id').agg({
        'consult_no':'count',
        'duration': 'sum',
        'comment':'sum',
        'like_level':lambda x: x[x >= 4].count(),
        'consult_month':lambda x:x.nunique(),
        'last_consult':'min'
    })
    
    #构建其他指标
    temp['consult_mcnts'] = temp['consult_no']/temp['consult_month'] #月均问诊次数
    temp['consult_mdurs'] = temp['duration']/temp['consult_month'] #月均问诊时长
    temp['consult_durs'] = temp['duration']/temp['consult_no'] #单次平均问诊时长
    temp['comment_ratio'] = temp['comment'] / temp['consult_no'] #评价率
    temp['favor_ration'] = temp['like_level'] / temp['comment'].sum() #好评率

    user_consult_stat = pd.DataFrame(df['user_id'].unique(), columns=['user_id'])

    user_consult_stat = pd.merge(user_consult_stat, temp, on='user_id', how='left')
    user_consult_stat.rename(columns={
        'consult_no':'consult_cnt',
    },inplace=True)

    return user_consult_stat

user_consult_mertics = consult_calcu(consult_df)
user_consult_mertics.head(3)

,user_id,consult_cnt,duration,comment,like_level,consult_month,last_consult,consult_mcnts,consult_mdurs,consult_durs,comment_ratio,favor_ration
0,570051393580120,172,1944.0,30,30,3,276,57.333333,648.0,11.302326,0.174419,0.000318
1,570051491040126,9,110.0,0,0,2,321,4.500000,55.0,12.222222,0.000000,0.000000
2,570050307940122,1,20.0,0,0,1,361,1.000000,20.0,20.000000,0.000000,0.000000


In [117]:
renew_user = user_consult_df[user_consult_df['is_renew'] == 1] #续费会员数据集
nonrenew_user = user_consult_df[user_consult_df['is_renew']==0] #非续费会员数据集

In [258]:
nonrenew_user.shape

(2, 9)

In [118]:
def process_consult_rename(df):
    df = consult_calcu(df)
    df.drop(['consult_month','comment','like_level'],axis=1,inplace=True) 
    df = df.rename(columns={
        'consult_cnt':'consult_cnt_prd',
        'duration': 'duration_prd',
        'consult_durs':'consult_durs_prd',
        'consult_mcnts':'consult_mcnts_prd',
        'consult_mdurs':'consult_mdurs_prd',
        'comment_ratio':'comment_ratio_prd',
        'favor_ration':'favor_ration_prd',
        'last_consult':'last_consult_prd',
    })
    return df

renew_user_period = renew_user[renew_user['consult_start_time'] < renew_user['renew_time']] #
renew_user_period = renew_user[renew_user['consult_start_time'] > renew_user['be_member_time']]

nonrenew_user_period = nonrenew_user[nonrenew_user['consult_start_time'] < nonrenew_user['last_day']]
nonrenew_user_period = nonrenew_user[nonrenew_user['consult_start_time'] > nonrenew_user['renew_time']]

In [119]:
renew_user = process_consult_rename(renew_user_period)
nonrenew_user = process_consult_rename(nonrenew_user_period)

In [120]:
renew_user_period.shape

(8465, 22)

In [121]:
renew_user.head(5)

,user_id,consult_cnt_prd,duration_prd,last_consult_prd,consult_mcnts_prd,consult_mdurs_prd,consult_durs_prd,comment_ratio_prd,favor_ration_prd
0,570026677930135,1,18.0,358,1.00,18.0,18.000000,0.000000,0.000000
1,570022650830127,7,84.0,204,1.75,21.0,12.000000,0.428571,0.001946
2,570038491870159,15,206.0,126,3.75,51.5,13.733333,0.333333,0.003243
3,570038542940137,3,64.0,352,3.00,64.0,21.333333,0.000000,0.000000
4,570004842750155,2,28.0,355,2.00,28.0,14.000000,0.000000,0.000000
